# Building a Cloud Service App with Cognitive Vision APIs
In this lab, we will be building a [Django](https://www.djangoproject.com/) Cloud Service App, and we will be deploying to 
[Azure](https://azure.microsoft.com/en-us/services/cloud-services/) which uses the 
[PaaS](https://en.wikipedia.org/wiki/Platform_as_a_service) paradigm of deployment and programming.

The advantage of using a [PaaS](https://en.wikipedia.org/wiki/Platform_as_a_service) instead of deploying
on [Virtual Machines](https://azure.microsoft.com/en-us/services/virtual-machines/)
\- which use [IaaS](https://en.wikipedia.org/wiki/Cloud_computing#Infrastructure_as_a_service_.28IaaS.29) paradigms \-,
is deployment becomes a lot easier.

In this hands on lab (HOL), we will create a cloud service app that allows users to upload photos, and then
utilizing [Cortana's Vision API](https://www.microsoft.com/cognitive-services/en-us/face-api),
we can sort the images into two containers: one with faces, and one without faces.

This HOL will expose us to Azure Cloud Storage, Azure Queues, Azure Cloud Services and Cortana Cognative
services as well.

# -1. App Planning

From a high level, the magic behind our app will look like this.

![](http://i.imgur.com/dmrNiDQ.png)

# 0. Prerequisites

You will need the following to complete this HOL

- [Visual Studio](https://www.visualstudio.com/en-us/visual-studio-homepage-vs.aspx) Installed
- [Azure Account](https://portal.azure.com/)

This lab will assume that you are familiar with Python, Django and the Azure dashboard.

# 1. Start a new project
Create a new Python Azure Cloud Service Project

![](https://github.com/qwergram/Python-Cortana-Vision-API-Guide/raw/master/images/createproject.png)

# 2. Select Worker and Web Roles
Add one worker and one web role

![](http://i.imgur.com/syWSZJ5.png)

# 3. Build Django Apps

Navigate to the solution explorer as shown before and add `Apps` to your current Django Project.

![](http://i.imgur.com/HCBtqkM.png)

Once you created `upload` and `library`, your directory structure should look something like this:
```
- Django Web Role\ (Root Directory)
    - Django_Web_Role\ (Project Directory)
        - __init__.py
        - settings.py
        - urls.py
        - wsgi.py
    - library\ (App Directory)
        - migrations\
        - templates\
        - __init__.py
        - admin.py
        - models.py
        - tests.py
        - views.py
    - upload\ (App Directory)
        - migrations\
        - templates\
        - __init__.py
        - admin.py
        - models.py
        - tests.py
        - views.py
    - manage.py
    - bin\ (Scripts for deployment)
        - *.ps1 (Some powershell scripts)
```

# 4. Update `Django_web_Role\settings.py` and `manage.py`

In `settings.py`, update `INSTALLED_APPS`. Append the new apps you created to it.
    
```python
INSTALLED_APPS = [
    # Add your apps here to enable them
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'library', # < New apps
    'upload',  # < New apps
]
```

# 5. Update `manage.py`

There is currently a bug in Visual studio where it sometimes doesn't populate `manage.py` properly. Navigate to the file `manage.py` make sure that the string `"$safeprojectname$"` does not exist and replace it with the proper library name.

```python
#!/usr/bin/env python
"""
Command-line utility for administrative tasks.
"""

import os
import sys

if __name__ == "__main__":
    os.environ.setdefault(
        "DJANGO_SETTINGS_MODULE",
        "Django_Web_Role.settings" # < make sure this is a valid python module
                                   # < NOT "$safeprojectname$.settings"
    )

    from django.core.management import execute_from_command_line

    execute_from_command_line(sys.argv)
```

# 6. Create routes

In the file `Django_Web_Role\urls.py`, update `urlpatterns` to use your new apps'  when routing.

```python

from django.conf.urls import include, url

# import Django_Web_Role.views                                  # < Delete this import

import upload.urls                                              # < New Import (Doesn't exist yet)
# import library.urls                                           # < New Import (Doesn't exist yet)

urlpatterns = [
    url(r'^<your_path_here>/', include(upload.urls)),           # < New route (Doesn't exist yet)
    # url(r'^<your_path_here>/', include(library.urls)),        # < New Route (Doesn't exist yet)
]
```

__Note__: You can set your path to be root (e.g. `http://example.com/`) with `url(r'^', include(<your_lib>.urls))`.